In [2]:
import io
import time
from sklearn import metrics
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
import tensorflow as tf
from sklearn import svm, datasets
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import log_loss

from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64

Using TensorFlow backend.


In [3]:
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

    # Plot a confusion matrix.
# cm is the confusion matrix, names are the names of the classes.
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted')

In [20]:
import pandas as pd 
path="/home/bsplab/Desktop/manikanta/16khz_data/NEWFRAME_SHIFTING_DATA/timedomain/250ms"
for chunk in pd.read_csv('finaltimedomain_250ms.csv',chunksize=14400):
    do_processing()
    train_algorithm

NameError: name 'do_processing' is not defined

In [16]:
import pandas as pd
import time
tic=time.time()
path="/home/bsplab/Desktop/manikanta/16khz_data/NEWFRAME_SHIFTING_DATA/timedomain/250ms"
chunksize=14400
#chunks = []
#for chunk in pd.read_csv('finaltimedomain_250ms.csv'):chunks.append(chunk)

#df = pd.contact(chunks,axis=0)
#df=pd.read_csv("/home/bsplab/Desktop/manikanta/16khz_data/NEWFRAME_SHIFTING_DATA/timedomain/250ms/finaltimedomain_250ms.csv",na_values=['NA','?'],error_bad_lines=False)
df.columns=['NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','Label']
filename_write = os.path.join(path,"/home/bsplab/Desktop/manikanta/16khz_data/NEWFRAME_SHIFTING_DATA/timedomain/250ms/out-of-sample_mySVM_2500ms_timedomain.csv")
df.head()
toc=time.time()
print(str(1000*(toc-tic))+"ms")




b'Skipping line 28801: expected 4001 fields, saw 8001\nSkipping line 28802: expected 4001 fields, saw 8001\nSkipping line 28803: expected 4001 fields, saw 8001\nSkipping line 28804: expected 4001 fields, saw 8001\nSkipping line 28805: expected 4001 fields, saw 8001\nSkipping line 28806: expected 4001 fields, saw 8001\nSkipping line 28807: expected 4001 fields, saw 8001\nSkipping line 28808: expected 4001 fields, saw 8001\nSkipping line 28809: expected 4001 fields, saw 8001\nSkipping line 28810: expected 4001 fields, saw 8001\nSkipping line 28811: expected 4001 fields, saw 8001\nSkipping line 28812: expected 4001 fields, saw 8001\nSkipping line 28813: expected 4001 fields, saw 8001\nSkipping line 28814: expected 4001 fields, saw 8001\nSkipping line 28815: expected 4001 fields, saw 8001\nSkipping line 28816: expected 4001 fields, saw 8001\nSkipping line 28817: expected 4001 fields, saw 8001\nSkipping line 28818: expected 4001 fields, saw 8001\nSkipping line 28819: expected 4001 fields, s

1455258.6348056793ms


In [17]:
np.random.seed(42)
df = df.reindex(np.random.permutation(df.index))
df.reset_index(inplace=True, drop=True)
df.columns

Index(['NEW', 'NEW', 'NEW', 'NEW', 'NEW', 'NEW', 'NEW', 'NEW', 'NEW', 'NEW',
       ...
       'NEW', 'NEW', 'NEW', 'NEW', 'NEW', 'NEW', 'NEW', 'NEW', 'NEW', 'Label'],
      dtype='object', length=4001)

In [18]:
X = df[['NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW',
        'NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW','NEW']]
y = df['Label']


MemoryError: 

In [ ]:
# Encode to a 2D matrix for training
Label=encode_text_index(df,'Label')
print("Labelling is:{}".format(Label))

In [13]:
# 25 % holdout data
main, holdout, main1, holdout1 = train_test_split(X, y, test_size=0.20,random_state = 0)

SyntaxError: invalid syntax (<ipython-input-13-d64e1b921604>, line 2)

In [ ]:
print("Shape of X : {}".format(X.shape))
print("Shape of y : {}".format(y.shape))
print("Shape of x_main : {}".format(x_main.shape))
print("Shape of x_holdout : {}".format(x_holdout.shape))
print("Shape of y_main : {}".format(y_main.shape))
print("Shape of y_holdout : {}".format(y_holdout.shape))

In [ ]:
# dividing X, y into train and test data
x_train, x_test, y_train, y_test = train_test_split(x_main, y_main, test_size=0.20,random_state = 0)

In [ ]:
print("Shape of x_train : {}".format(x_train.shape))
print("Shape of x_test : {}".format(x_test.shape))
print("Shape of y_train : {}".format(y_train.shape))
print("Shape of y_test : {}".format(y_test.shape))

In [ ]:
# preprocessing of training data
scaler = preprocessing.StandardScaler().fit(x_train)
#scaler

In [13]:
X_train = scaler.transform(x_train)
X_holdout = scaler.transform(x_holdout) 
# preprocessing of testing data
X_test= scaler.transform(x_test) 

NameError: name 'scaler' is not defined

In [ ]:
from sklearn.externals import joblib
scaler_file = "my_scaler_100ms.save"
joblib.dump(scaler, scaler_file) 

In [ ]:
## RBF kernel training
#### Training
tic=time.time()
svclassifier_rbf = SVC(kernel='rbf',C=1, max_iter=-1,verbose=True,probability=True) 
svclassifier_rbf.fit(X_train, y_train)
#scores = cross_val_score( svclassifier_rbf, X_train, y_train, cv=5,scoring='f1_macro')
toc=time.time()
print(str(1000*(toc-tic))+"ms")

In [ ]:
## Testing

tic=time.time()
y_rbf = svclassifier_rbf.predict(X_test)
toc=time.time()
print(str(1000*(toc-tic))+"ms")

In [ ]:
# model accuracy for X_test  
accuracy = accuracy_score(y_test, y_rbf)
print (accuracy)

In [ ]:
## Evaluation of Algorithm
print(confusion_matrix(y_test, y_rbf))  
print(classification_report(y_test, y_rbf))

In [ ]:
# save the model to disk
filename = 'SVM_100ms_Rbf_model.save'
joblib.dump(svclassifier_rbf, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model_rbf = joblib.load(open(filename, 'rb'))

In [ ]:
tic=time.time()
holdout_pred_rbf = loaded_model_rbf.predict(X_holdout)
toc=time.time()
print(str(1000*(toc-tic))+"ms")

In [ ]:
rbf_score = accuracy_score(y_holdout, holdout_pred_rbf)
print("Holdout accuracy with rbf kernel is: {}".format(rbf_score))

In [ ]:
## Turn off the scintific notation 
np.set_printoptions(suppress=True)

In [ ]:
cm = confusion_matrix(y_holdout, holdout_pred_rbf)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm, Label)
plt.savefig('cm_holdout_mySVM_100ms_mani_rbf.png',dpi=150)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_holdout, holdout_pred_rbf)
np.set_printoptions(precision=2)


# Plot non-normalized confusion matrix
plt.figure(figsize=(5,3))
plot_confusion_matrix(cnf_matrix, classes=['ac&fan', 'crying','music', 'speech'],
                      title='Confusion matrix')

plt.savefig('mani1.png')

In [ ]:
print(classification_report(y_holdout, holdout_pred_rbf))

In [ ]:
model_prob_rbf = loaded_model_rbf.predict_proba(X_holdout) ##need prob for getting logloss
rbf_log_loss = log_loss(y_holdout, model_prob_rbf)
print("Log loss score of Holdout data for RBF kernel: {}".format(rbf_log_loss))